# Protrein generative models evaluation metrics
## Environment Preparation
### Conda Environment

You can set up the conda environment by running the following command:

In [1]:
conda env create -f eval.yml

^C

Note: you may need to restart the kernel to use updated packages.


Besides, you have to install the following packages:

``` bash

pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
pip install fair-esm


```

### Foldseek database

When we calculate the novelty metric, we use the Foldseek database.


``` bash

conda install -c conda-forge -c bioconda foldseek
mkdir ./foldseek_database
cd ./foldseek_database
foldseek databases PDB pdb tmp 

```

Foldseek will download the PDB database automatically. After the download, you directory should look like this:

```
foldseek_database
    ├── pdb
    ├── pdb_ca
    ├── pdb_ca.dbtype
    ├── pdb_ca.index
    ├── pdb_clu
    ├── pdb_clu.dbtype
    ├── pdb_clu.index
    ├── pdb.dbtype
    ├── pdb_h
    ├── pdb_h.dbtype
    ├── pdb_h.index
    ├── pdb.index
    ├── pdb.lookup
    ├── pdb_mapping
    ├── pdb_seq.0 -> pdb
    ├── pdb_seq.1
    ...
```

After downloading the foldseek database, you need to replace the database path in the `foldseek_database` field of the `configs/evaluation.yaml` file.

### Maxcluster

When we cluster the designed protein based on their structure, we use maxcluster to cluster them.

``` bash
wget https://www.sbg.bio.ic.ac.uk/maxcluster/maxcluster64bit
```

#### Example data

We provide some example data `./example_data` for testing purposes.

```
└── length_70
    ├── sample_0
    │   ├── bb_traj.pdb
    │   ├── sample.pdb
    │   └── x0_traj.pdb
    ├── sample_1
    │   ├── bb_traj.pdb
    │   ├── sample.pdb
    │   └── x0_traj.pdb
```



### ProteinMPNN

We can use the ProteinMPNN model to design a sequence for a given structure. 

``` bash

git clone https://github.com/dauparas/ProteinMPNN.git

```

## Evaluation

### Single pdb evaluation

In [2]:
# import package
import os
import time
import numpy as np
import hydra
import torch
import subprocess
import logging
import pandas as pd
import shutil
from datetime import datetime
from biotite.sequence.io import fasta
import GPUtil
from typing import Optional, Union, List
from analysis import utils as au
from analysis import metrics
from data import utils as du
from omegaconf import DictConfig, OmegaConf
from openfold.data import data_transforms
import esm
from pathlib import Path
import mdtraj as md
from openfold.np import residue_constants
from tmtools import tm_align
from openfold.utils.superimposition import superimpose
from tqdm import tqdm
import re
import yaml

In [3]:
class DictToObject:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                setattr(self, key, DictToObject(value))
            else:
                setattr(self, key, value)

def load_config(file_path):
    with open(file_path, "r") as f:
        config = yaml.safe_load(f)
    return config


In [ ]:
from EvalRunner import EvalRunner
config_dict = load_config("./configs/evaluation.yaml")
conf = DictToObject(config_dict)
EvalModel = EvalRunner(conf)

# Warning: ESMFold and ProteinMPNN pipeline need more than 12G GPU memory


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.79 GiB total capacity; 5.32 GiB already allocated; 12.19 MiB free; 5.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

#### Self consistency metrics

```mermaid

graph TD;
    A[Protein Generative models] --> B[ProteinMPNN (inverse folding)];
    B --> C[ESMFold (folding)];

```


In [5]:
# Example pdb path
pdb_path = "/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_0/"
sc_output_dir = os.path.join(pdb_path, "self_consistency")
os.makedirs(sc_output_dir, exist_ok=True)

sc_results = EvalModel.calc_designability(sc_output_dir, pdb_path)
sc_results

,tm_score,bb_rmsd,sample_path,header,sequence
0,0.285942,12.353213,/home/shuaikes/server2/shuaikes/projects/prote...,"sample, score=2.0021, global_score=2.0021, fix...",AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,0.820439,2.127891,/home/shuaikes/server2/shuaikes/projects/prote...,"T=0.1, sample=1, score=1.0243, global_score=1....",ATLTKMLVKVKDKSITLEDVKKIIKEVGVDAEIEIDKETNTVTITA...
2,0.838250,2.160685,/home/shuaikes/server2/shuaikes/projects/prote...,"T=0.1, sample=2, score=1.0038, global_score=1....",SILTKLKIKIKDKSINLEDIKKIAKEEGINCKIEIDKETNEVIVEA...
3,0.852542,1.917273,/home/shuaikes/server2/shuaikes/projects/prote...,"T=0.1, sample=3, score=0.9787, global_score=0....",MVKTKMKIKIKDKSINKEDIEKIVKEEGLNVEIEIDKDTNTVTVKG...


#### Sub-structure ratio evaluation

In [6]:
path = os.path.join(pdb_path, "sample.pdb")
sub_ratio = EvalModel.calc_mdtraj_metrics(path)
sub_ratio

{'non_coil_percent': 0.6571428571428571,
 'coil_percent': 0.34285714285714286,
 'helix_percent': 0.34285714285714286,
 'strand_percent': 0.3142857142857143,
 'radius_of_gyration': 1.0974538862859071}

#### Novelty: pdbTM

In [ ]:
path = os.path.join(pdb_path, "sample.pdb")
value = EvalModel.pdbTM(pdb_path)
value

TM-Score between sample.pdb and its closest protein in PDB is 0.867.


* Maybe you will encounter some issues here because of the different versions of the packages and foldseek path. So we provide a shell script to help you to run the foldseek to calculate the pdbTM.

In [10]:
!foldseek -h

Foldseek enables fast and sensitive comparisons of large structure sets. It reaches sensitivities similar to state-of-the-art structural aligners while being at least 20,000 times faster.

Please cite:
van Kempen, M., Kim, S.S., Tumescheit, C., Mirdita, M., Lee, J., Gilchrist, C.L.M., Söding, J., and Steinegger, M. Fast and accurate protein structure search with Foldseek. Nature Biotechnology, doi:10.1038/s41587-023-01773-0 (2023)

foldseek Version: 9.427df8a
© Michel van Kempen, Stephanie Kim, Charlotte Tumescheit, Milot Mirdita, Jeongjae Lee, Cameron L. M. Gilchrist, Johannes Söding, Martin Steinegger

usage: foldseek <command> [<args>]

Easy workflows for plain text input/output
  easy-search       	Structual search
  easy-cluster      	Slower, sensitive clustering
  easy-rbh          	Find reciprocal best hit
  easy-multimersearch	Complex level search
  easy-complexsearch	

Main workflows for database input/output
  createdb          	Convert PDB/mmCIF/tar[.gz]/DB files or director

In [44]:
#!/bin/bash

# variables
input="/home/shuaikes/Project/protein-evaluation-notebook/example_data/length_70/sample_0/sample.pdb"
foldseek_database_path="/home/shuaikes/Project/protein-evaluation-notebook/foldseek_database/pdb"
output_file="/home/shuaikes/Project/protein-evaluation-notebook/example_data/length_70/sample_0/sample_pdb.m8"
tmp_path="./tmp/"
# replace with your actual input file path and output file path


# set up directory
!echo "Running Foldseek..."
!mkdir -p $tmp_path


# run the command
!foldseek easy-search \
    $input \
    $foldseek_database_path \
    $output_file \
    $tmp_path \
    --format-mode 4 \
    --format-output query,target,evalue,alntmscore,rmsd,prob \
    --alignment-type 1 \
    --num-iterations 2 \
    -e inf \
    -v 0

Running Foldseek...
rmdb ./tmp//7846871783882344137/search_tmp/8537189719342743872/pref_tmp_1 

Time for processing: 0h 0m 0s 0ms
mergedbs ./tmp//7846871783882344137/search_tmp/8537189719342743872/profile_0 ./tmp//7846871783882344137/result ./tmp//7846871783882344137/search_tmp/8537189719342743872/aln_0 ./tmp//7846871783882344137/search_tmp/8537189719342743872/aln_tmp_1 

Merging the results to ./tmp//7846871783882344137/result
[=================================================================] 100.00% 1 eta -
Time for merging to result: 0h 0m 0s 0ms
Time for processing: 0h 0m 0s 0ms
rmdb ./tmp//7846871783882344137/search_tmp/8537189719342743872/aln_0 

Time for processing: 0h 0m 0s 0ms
rmdb ./tmp//7846871783882344137/search_tmp/8537189719342743872/aln_tmp_1 

Time for processing: 0h 0m 0s 0ms


In [17]:
# replace with your own output file
output_file = "/home/shuaikes/Project/protein-evaluation-notebook/example_data/length_70/sample_0/sample_pdb.m8"
result = pd.read_csv(output_file, sep="\t")
top_pdbTM = round(result["alntmscore"].head(1).max(), 3)
top_pdbTM

0.867

#### Calculate all metrics

In [9]:
EvalModel.calc_all_metrics(sc_output_dir, pdb_path)

Calculating all metrics...
##########################################################################
Calculating designability...
   tm_score    bb_rmsd                                        sample_path  \
0  0.285942  12.353213  /home/shuaikes/server2/shuaikes/projects/prote...   
1  0.820439   2.127891  /home/shuaikes/server2/shuaikes/projects/prote...   
2  0.838250   2.160685  /home/shuaikes/server2/shuaikes/projects/prote...   
3  0.852542   1.917273  /home/shuaikes/server2/shuaikes/projects/prote...   

                                              header  \
0  sample, score=2.0021, global_score=2.0021, fix...   
1  T=0.1, sample=1, score=1.0243, global_score=1....   
2  T=0.1, sample=2, score=1.0038, global_score=1....   
3  T=0.1, sample=3, score=0.9787, global_score=0....   

                                            sequence  
0  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...  
1  ATLTKMLVKVKDKSITLEDVKKIIKEVGVDAEIEIDKETNTVTITA...  
2  SILTKLKIKIKDKSINLEDIKKIAKEEGINCKIE

#### Diversity: number of clusters

We are not able to calculate the diversity of the cluster for just single protein strucuture. So we need multiple pdb files here and we use csv to specify the path the protein structures.

Your csv file should like this:

```
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_0/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_1/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_2/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_3/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_4/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_5/
```

In [ ]:
pdb_csv_path = "/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/pdb_path.csv"
clusters = EvalModel.calc_diversity(pdb_csv_path)
clusters

6

* Also, you may encounter similar issues with other libraries or tools. So we provide some scripts to run the clustering.

In [30]:
# set up workspace

pdb_csv_path = "/home/shuaikes/Project/protein-evaluation-notebook/pdb_path.csv"
df = pd.read_csv(pdb_csv_path, header=None)
pdb_path_list = df[0].tolist()

cluster_dir = os.path.join(Path(pdb_csv_path).parent, "cluster")
os.makedirs(cluster_dir, exist_ok=True)
designable_paths = pdb_path_list
designable_file_path = os.path.join(cluster_dir, "designable_paths.txt")
updated_paths = [os.path.join(path, "sample.pdb") for path in designable_paths]
with open(designable_file_path, "w") as f:
    f.write("\n".join(updated_paths))
    
print("designable_file_path: ", designable_file_path)
print("cluster_dir: ", cluster_dir)


designable_file_path:  /home/shuaikes/Project/protein-evaluation-notebook/cluster/designable_paths.txt
cluster_dir:  /home/shuaikes/Project/protein-evaluation-notebook/cluster


In [33]:
designable_file_path = "/home/shuaikes/Project/protein-evaluation-notebook/cluster/designable_paths.txt"
cluster_dir = "/home/shuaikes/Project/protein-evaluation-notebook/cluster"

!./maxcluster64bit -l $designable_file_path "$cluster_dir/all_by_all_lite" -C 2 -in -Rl "$cluster_dir/tm_results.txt" -Tm 0.5 > "$cluster_dir/output.txt"

In [ ]:
output_file = "/home/shuaikes/Project/protein-evaluation-notebook/cluster/output.txt"
designable_dir = "/home/shuaikes/Project/protein-evaluation-notebook/cluster/"

# open the output file
with open(output_file, "r") as f:
    stdout = f.read()

"INFO  : Reading PDB list file '/home/shuaikes/Project/protein-evaluation-notebook/cluster/designable_paths.txt'\nINFO  : Successfully read 0 PDBs\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08INFO  : Successfully read 6 / 6 PDBs from list file '/home/shuaikes/Project/protein-evaluation-notebook/cluster/designable_paths.txt'\nINFO  : Successfully read 6 Chain structures\nINFO  : Processed 0 of 15 MAXSUBs\nINFO  : CPU time = 0.02 seconds\nINFO  : Printing MaxSub output file /home/shuaikes/Project/protein-evaluation-notebook/cluster/tm_results.txt\nINFO  : ======================================\nINFO  : 3Djury (Threshold: >    20 pairs @ > 0.200)\nINFO  : ======================================\nINFO  : Rank     Model    Pairs       File\nINFO  :     1 :        1      136       /home/shuaikes/Project/protein-evaluation-notebook/example_data/length_70/sample_0/sample.pdb\nINFO  :     2 :        2      128     

In [43]:
# Extract number of clusters
match = re.search(
    r"INFO\s*:\s*(\d+)\s+Clusters\s+@\s+Threshold\s+(\d+\.\d+)\s+\(\d+\.\d+\)",
    stdout,
)
clusters = int(match.group(1))
cluster_results_path = os.path.join(designable_dir, "cluster_results.txt")
with open(cluster_results_path, "w") as f:
    f.write(stdout)

# Extract cluster centroids
cluster_lines = stdout.split("\n")
centroid_section = False
for line in cluster_lines:
    if "Centroids" in line:
        centroid_section = True
    if centroid_section:
        match = re.search(r"(?<=\s)(\/[^\s]+\.pdb)", line)
        if match is not None:
            centroid_path = match.group(1)
            copy_name = centroid_path.split("/")[-2] + ".pdb"
            shutil.copy(centroid_path, os.path.join(designable_dir, copy_name))

print("Number of decoys in cluster:", clusters)

Number of decoys in cluster: 6
